# Задания по SQL

In [ ]:
!pip install -r requirements.txt

In [12]:
import os
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")
load_dotenv()

True

Настраиваем psycopg

In [13]:
import psycopg2

psycopg_connection = psycopg2.connect(
    database=os.environ.get("POSTGRES_DB"),
    user=os.environ.get("POSTGRES_USER"),
    password=os.environ.get("POSTGRES_PASSWORD"),
    host=os.environ.get("POSTGRES_HOST"),
    port=os.environ.get("POSTGRES_PORT"),
)

Импортируем настройки SQLAlchemy

In [2]:
from sqlalchemy_connection import get_session, Movie, MovieImdb, UserMovieRating, UserMovieTag

sqlalchemy_session = next(get_session())

## Примеры

1. Выдача фильма по id

In [17]:
# Чистый SQL
query = "SELECT * FROM movie WHERE id = %s"
with psycopg_connection.cursor() as cursor:
    cursor.execute(query, (3417,))
    movie_example = cursor.fetchone()
    print(f"id={movie_example[0]}, title={movie_example[1]}, genres={movie_example[2]}")

id=3417, title=Crimson Pirate, The (1952), genres=Adventure|Comedy


In [6]:
# SQLAlchemy
movie_example = sqlalchemy_session.query(Movie).get(3417)
movie_example

Movie(id=3417, title=Crimson Pirate, The (1952), genres=Adventure|Comedy)

2. Выдача 10 наибольших оценок

In [18]:
# Чистый SQL
query = "SELECT * FROM user_movie_rating ORDER BY rating DESC LIMIT 10"
with psycopg_connection.cursor() as cursor:
    cursor.execute(query)
    most_ratings = cursor.fetchall()
    for most_rating in most_ratings:
        print(f"movie_id={most_rating[0]}, user_id={most_rating[1]}, rating={most_rating[2]}, timestamp={most_rating[3]}")

movie_id=12, user_id=216, rating=1, timestamp=5.0
movie_id=16, user_id=260, rating=1, timestamp=5.0
movie_id=11, user_id=163, rating=1, timestamp=5.0
movie_id=5, user_id=50, rating=1, timestamp=5.0
movie_id=14, user_id=231, rating=1, timestamp=5.0
movie_id=7, user_id=101, rating=1, timestamp=5.0
movie_id=4, user_id=47, rating=1, timestamp=5.0
movie_id=9, user_id=151, rating=1, timestamp=5.0
movie_id=10, user_id=157, rating=1, timestamp=5.0
movie_id=19, user_id=333, rating=1, timestamp=5.0


In [8]:
# SQLAlchemy
most_ratings = sqlalchemy_session.query(UserMovieRating).order_by(UserMovieRating.rating.desc()).limit(10).all()
for most_rating in most_ratings:
    print(most_rating)

UserMovieRating(movie_id=216, user_id=1, rating=5.0, timestamp=964981208)
UserMovieRating(movie_id=260, user_id=1, rating=5.0, timestamp=964981680)
UserMovieRating(movie_id=163, user_id=1, rating=5.0, timestamp=964983650)
UserMovieRating(movie_id=50, user_id=1, rating=5.0, timestamp=964982931)
UserMovieRating(movie_id=231, user_id=1, rating=5.0, timestamp=964981179)
UserMovieRating(movie_id=101, user_id=1, rating=5.0, timestamp=964980868)
UserMovieRating(movie_id=47, user_id=1, rating=5.0, timestamp=964983815)
UserMovieRating(movie_id=151, user_id=1, rating=5.0, timestamp=964984041)
UserMovieRating(movie_id=157, user_id=1, rating=5.0, timestamp=964984100)
UserMovieRating(movie_id=333, user_id=1, rating=5.0, timestamp=964981179)


3. Выдача 10 наибольших оценок с фильмами

In [20]:
# Чистый SQL
query = """
SELECT
    movie.title AS movie_title,
    movie.genres AS movie_genres,
    umr.user_id AS user_id,
    umr.rating AS rating
FROM movie
JOIN user_movie_rating umr on movie.id = umr.movie_id
ORDER BY umr.rating DESC
LIMIT 10
"""
with psycopg_connection.cursor() as cursor:
    cursor.execute(query)
    most_ratings_and_films = cursor.fetchall()
    for item in most_ratings_and_films:
        print(f"Movie: {item[0]}, genres: {item[1]}, user_id: {item[2]}, rating: {item[3]}")

Movie: Billy Madison (1995), genres: Comedy, user_id: 1, rating: 5.0
Movie: Star Wars: Episode IV - A New Hope (1977), genres: Action|Adventure|Sci-Fi, user_id: 1, rating: 5.0
Movie: Desperado (1995), genres: Action|Romance|Western, user_id: 1, rating: 5.0
Movie: Usual Suspects, The (1995), genres: Crime|Mystery|Thriller, user_id: 1, rating: 5.0
Movie: Dumb & Dumber (Dumb and Dumber) (1994), genres: Adventure|Comedy, user_id: 1, rating: 5.0
Movie: Bottle Rocket (1996), genres: Adventure|Comedy|Crime|Romance, user_id: 1, rating: 5.0
Movie: Seven (a.k.a. Se7en) (1995), genres: Mystery|Thriller, user_id: 1, rating: 5.0
Movie: Rob Roy (1995), genres: Action|Drama|Romance|War, user_id: 1, rating: 5.0
Movie: Canadian Bacon (1995), genres: Comedy|War, user_id: 1, rating: 5.0
Movie: Tommy Boy (1995), genres: Comedy, user_id: 1, rating: 5.0


In [9]:
# SQLAlchemy
most_ratings_and_films = sqlalchemy_session.query(
    Movie.title.label("movie_title"),
    Movie.genres.label("movie_genres"),
    UserMovieRating.user_id.label("user_id"),
    UserMovieRating.rating.label("rating")
).select_from(Movie) \
    .join(UserMovieRating, Movie.id == UserMovieRating.movie_id) \
    .order_by(UserMovieRating.rating.desc()) \
    .limit(10) \
    .all()

for item in most_ratings_and_films:
    print(f"Movie: {item.movie_title}, genres: {item.movie_genres}, user_id: {item.user_id}, rating: {item.rating}")

Movie: Billy Madison (1995), genres: Comedy, user_id: 1, rating: 5.0
Movie: Star Wars: Episode IV - A New Hope (1977), genres: Action|Adventure|Sci-Fi, user_id: 1, rating: 5.0
Movie: Desperado (1995), genres: Action|Romance|Western, user_id: 1, rating: 5.0
Movie: Usual Suspects, The (1995), genres: Crime|Mystery|Thriller, user_id: 1, rating: 5.0
Movie: Dumb & Dumber (Dumb and Dumber) (1994), genres: Adventure|Comedy, user_id: 1, rating: 5.0
Movie: Bottle Rocket (1996), genres: Adventure|Comedy|Crime|Romance, user_id: 1, rating: 5.0
Movie: Seven (a.k.a. Se7en) (1995), genres: Mystery|Thriller, user_id: 1, rating: 5.0
Movie: Rob Roy (1995), genres: Action|Drama|Romance|War, user_id: 1, rating: 5.0
Movie: Canadian Bacon (1995), genres: Comedy|War, user_id: 1, rating: 5.0
Movie: Tommy Boy (1995), genres: Comedy, user_id: 1, rating: 5.0


3. Подсчет количества записей о фильмах

In [22]:
# Чистый SQL
query = "SELECT COUNT(*) FROM movie"
with psycopg_connection.cursor() as cursor:
    cursor.execute(query)
    movie_count = cursor.fetchone()
    print(f"Количество записей о фильмах: {movie_count[0]}")

Количество записей о фильмах: 9742


In [10]:
# SQLAlchemy
movie_count = sqlalchemy_session.query(Movie).count()
print(f"Количество записей о фильмах: {movie_count}")

Количество записей о фильмах: 9742


## Задания:

1. Выдайте топ 10 фильмов с наибольшей средней оценкой

2. Найдите все фильмы, у которых нет или индекса IMDB, или TMDB

3. Найдите все фильмы, в названии которых употребляется слово `pirate` (регистр не имеет значение).

4. Найдите теги, проставленные пользователями, которые употребляются наиболее часто

5. Найдите фильм с наибольшим количеством тегов (выведите название фильма и список тегов)

6. Найдите все фильмы, для которых нет ни одной оценки

7. Найдите пользователей, которые чаще всего оставляли отзывы. Для таких пользователей найдите: среднюю, минимальную и
максимальную оценки

8. Из названия вычлените год и для каждого найдите количество фильмов. Выведите топ 10 годов, за которые выпустили
наибольшее количество фильмов

9. Для каждого года, найдите среднюю оценку фильмов. Изобразите соответствующий график изменений

10. В таблице `uset_movie_rating` найдите топ 10 годов, за которые пользователи оставляли свои рейтинги

11. Добавьте несколько новых фильмов в таблицу `movie`

12. Измените у любой из записи какое-либо поле

13. Удалите из базы данных фильм, у которого есть оценки от пользователей и теги

14. (Сложно) Для оценок по годам найдите моду, медиану, 25-й и 75-й перцентили.